## Tutorial on how to use inetmodels python API

### Main Website: http://inetmodels.com

1. Download inetmodels.py file from the github repository/the main website

2. Import the library

```python
from inetmodels import *
```
3. Initialize the Inetmodels class. It will also show the network type and categories that are available

```python
k=Inetmodels()
```

4. Check the specific network that you want

```python
k.printCategories(networkType,categoryType)
```

5. Query the genes/analytes that you need

```python
k.query(networkType, # MON or GCN, default = ""
      categoryType, # integer, default = 0
      categoryName, # string, default = ""
      search, #list of strings, default = []
      pruning, #float, default = 2.5, means FDR < 10E-2.5
      nodeLimit, #integer, default = 10 
      firstNeighbour, #boolean, default = True 
      gene, #boolean, default = True 
      metabolite, #boolean, default = True 
      protein, #boolean, default = True 
      clinical, #boolean, default = True 
      microbiome #boolean, default = True 
       )
```

In [ ]:
from inetmodels import *

In [ ]:
#Initialization
k=Inetmodels()

In [ ]:
#Print Categories for Multi-Omics Network, categoryType SCAPIS - SciLifeLab Wellness Profiling Study
k.printCategories('MON',1)

In [ ]:
#Print Categories for Gene Co-Expression Network, categoryType Cancer
k.printCategories('GCN',2)

#k.Categories are the dictionary of all network categories

In [ ]:
#Querying "Weight" and "L-Leucine " in P100 Study
k.query('MON',3,'Longitudinal - Overall',search = ['Weight','L-Leucine'])

In [ ]:
#Edges of the network
k.edges

In [ ]:
#Nodes of the network. "index" is renamed to "node"
k.nodes.rename_axis('node')

In [ ]:
#Querying "FASN" and "PKLR" in Liver Cancer
k.query('GCN',2,'Liver (LIHC)',search = ['FASN','PKLR'])

In [ ]:
#Edges of the network
k.edges

In [ ]:
#Nodes of the network
k.nodes

In [ ]:
## Example on the help page
#Querying "L-Serine", "FGF21", and "ALAT" in SCAPIS - SciLifeLab Wellness Profiling Study, and only showing clinical variables and microbiomes neighbor of those input variables
#pruning variable to 0 and nodeLimit to 10
k.query('MON',1,'Longitudinal - Overall',
        search = ['L-Serine','FGF21', 'ALAT'], 
        gene = False, 
        metabolite= False, 
        protein = False, 
        nodeLimit = 10, 
        pruning = 0)

In [ ]:
#Edges of the network
k.edges

In [ ]:
#Nodes of the network
k.nodes

## iGraph Integration

We next build a network based on the edges above using iGraph package (has to be installed first, not part of the package).

In [ ]:
import igraph, re

In [ ]:
# create network object
network = igraph.Graph.TupleList([tuple(x) for x in k.edges[['Source','Target','Weight']].values], directed=False, edge_attrs=['weight'])

In [ ]:
# add node parameters
for node_i in network.vs['name']:
    network.vs.find(node_i)['analyte_type'] = k.nodes.loc[node_i,'AnalyteType'] # add ANALYTE type to each node
    network.vs.find(node_i)['short_name'] = re.sub('\s\(.+','',node_i) # add short name to each node

In [ ]:
layout = network.layout_fruchterman_reingold() #create layout

#color for analyte type
node_color_dictionary = {'CLINICAL':'#FF6699','METABOLITE':'#CC0066','MICROBIOME':'#9999FF', 'PROTEIN':'#00CC99'}
node_color_list = [node_color_dictionary[x] for x in network.vs['analyte_type']]
edge_color_list = ['orange' if i < 0 else "blue" for i in network.es['weight']]

igraph.plot(network, layout=layout, vertex_color=node_color_list, edge_color=edge_color_list, vertex_label=network.vs['short_name'])